In [4]:
import os
os.chdir(os.path.dirname(os.path.abspath("__file__")))


In [2]:
import sys
print(sys.executable)
print(sys.path)

c:\Git_Tutorials\IntentClassification\.venv\Scripts\python.exe
['C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Git_Tutorials\\IntentClassification\\.venv', '', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages\\win32', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages\\Pythonwin']


In [1]:
import pandas as pd
import numpy as np
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
from pathlib import Path

In [4]:
# code to convert dataframe records to json
def csv_to_json_converter(csv_file_path):
    try:
        local_df = pd.read_csv(csv_file_path)
        local_df.reset_index(drop=True, inplace=True)
        records = local_df.to_dict(orient='records')
        return records
    except Exception as e:
        raise e

In [5]:
#code to insert the jsons into mongodb

def insert_data_mongodb(json_data, database, collection):
    try:
        #connect to MongoDB
        load_dotenv()
        client = MongoClient(os.getenv('MONGO_DB_URL'), server_api=ServerApi('1'))
        client.admin.command('ping')
        print("Connected to MongoDB")
        db = client[database]
        colln = db[collection]
        #insert the json data
        colln.insert_many(json_data)
        return(f"Inserted {len(json_data)} records into {collection} collection in {database} database.")
    except Exception as e:
        raise e

In [ ]:
# import src.IntentClassification.constants as const
# csv_file_path = const.LOCAL_DATASET_PATH
# database = "intent_classification"
# collection = "intents"

# json_data = csv_to_json_converter(csv_file_path)
# result = insert_data_mongodb(json_data, database, collection)
# print(result)

Inserted 7925 records into intents collection in intent_classification database.


In [5]:
def export_collection_as_csv(csv_file_path, database, collection):
    try:
        load_dotenv()
        client = MongoClient(os.getenv('MONGO_DB_URL'), server_api=ServerApi('1'))
        client.admin.command('ping')
        print("Connected to MongoDB")
        db = client[database]
        colln=db[collection]
        
        df = pd.DataFrame(list(colln.find()))
        print("export successful")
        if "_id" in df.columns:
            df.drop(columns=['_id'], inplace =True, axis=1)
        if "Unnamed: 0" in df.columns:
            df.drop(columns=['Unnamed: 0'], inplace =True, axis=1)
        print("csv_file saved succesfully")
        df.to_csv(csv_file_path, index=False, header = True)

    except Exception as e:
        print(e)
        raise e

In [6]:
from  src.IntentClassification import constants as const
csv_file_path = const.LOCAL_DATASET_PATH
database = const.MONGO_DB_DATABASE
collection = const.MONGO_DB_COLLECTION
export_collection_as_csv(csv_file_path, database, collection)



Connected to MongoDB
export successful
csv_file saved succesfully


In [ ]:
db = 
print(db)

intent_classification
